In [14]:
import requests
import pandas as pd
from io import StringIO


In [26]:
base_url = 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01003_001E,B01003_001M'
tract_level = '&for=tract:*&in=county:051&in=state:41'

In [44]:
def format_response(response_content):
    as_str = response_content.decode('utf-8')
    as_str_clean = as_str.replace('[','').replace(']','')
    tabular = pd.read_csv(StringIO(as_str_clean))
    tabular = tabular.rename(columns={'B01003_001E': 'population_estimate',
                                       'B01003_001M': 'population_margin'})
    tabular = tabular.iloc[:, :-1]  # last column is empty
    return tabular

In [45]:
response_tracts = requests.get(f'{base_url}{tract_level}')
tract_pop = format_response(response_tracts.content)

In [47]:
tract_list = tract_pop['tract'].tolist()

In [74]:
def make_url(base_url, tract_num):
    suffix = '&for=block%20group:*&in=tract:{:06d}&in=county:051&in=state:41'.format(tract_num)
    return f'{base_url}{suffix}'

In [80]:
# Loop over all tracts to get block group level data
block_group_pop = pd.DataFrame()
for tract_number in tract_list:
    print(f'Getting data for tract_number: {tract_number}')
    response = requests.get(make_url(base_url, tract_number))
    block_group_single = format_response(response.content)
    block_group_pop = pd.concat((block_group_pop, block_group_single),axis=0)

Getting data for tract_number: 6300
Getting data for tract_number: 2701
Getting data for tract_number: 3803
Getting data for tract_number: 4601
Getting data for tract_number: 6402
Getting data for tract_number: 7000
Getting data for tract_number: 7201
Getting data for tract_number: 3802
Getting data for tract_number: 3901
Getting data for tract_number: 5500
Getting data for tract_number: 5800
Getting data for tract_number: 5900
Getting data for tract_number: 6701
Getting data for tract_number: 7300
Getting data for tract_number: 8002
Getting data for tract_number: 8901
Getting data for tract_number: 9000
Getting data for tract_number: 9202
Getting data for tract_number: 9301
Getting data for tract_number: 9801
Getting data for tract_number: 8400
Getting data for tract_number: 8902
Getting data for tract_number: 9101
Getting data for tract_number: 10002
Getting data for tract_number: 7600
Getting data for tract_number: 7700
Getting data for tract_number: 8100
Getting data for tract_numb

In [81]:
tract_pop.to_csv('./tract_population.csv', index=False)
block_group_pop.to_csv('./block_group_population.csv', index=False)